In [1]:
(ns trenwand.notebook.orders
    (:require [clj-time.core :as dt]
              [clj-time.coerce :as dtc]
              [clj-time.format :as dtf]
              [clj-time.predicates :as dtp]
              [cuerdas.core :as str]
              [cheshire.core :as json]
              [trennwand.data :as data]
              [trennwand.order :as order]
              [trennwand.utils :as u]
              [net.cgrand.xforms :as x]
              [kixi.stats.core :as kixi]
              [criterium.core :refer [quick-bench]]
              [clojupyter.misc.display :as display]
              [clojupyter.misc.helper :as helper]))

# Zamówienie Teas

In [3]:
(u/vmap->table
    [[:product/name :nazwa] [:product/plu :plu] [:state/current :stan] 
     [:state/optimal :optymalny] [:sales/pace :tempo] [:order/qty :zamówić]]
    (order/generate-order-map :dc1))

nazwa,plu,stan,optymalny,tempo,zamówić
PRYMUS Słonecznik Biały turecki 80gx50sz,5907641483078,1.0,61.0,6.11,60.0
Chrupki Arachidowe solo 70g,5904918000204,2.0,52.0,5.17,50.0
PASZTET PROCHOWICKI Z DROBIU 160g,5901081000031,0.0,33.0,3.28,33.0
PRYMUS Słonecznik Rodzinny 220gx23szt,5907641483092,7.0,36.0,3.6,29.0
"NAPÓJ PEPSI COLA 2,25l",5900497112383,16.0,38.0,3.78,22.0
"WOJCIESZOWIANKA 1,5l Woda L/Gaz",5905453000339,25.0,46.0,4.64,21.0
JOGURT ZOTT NATURALNY B/CUK.180G,5906040063089,25.0,44.0,4.86,19.0
"WOJCIESZOWIANKA 1,5l W/Gaz",5905453000032,12.0,31.0,3.13,19.0
NAPÓJ LIPTON BRZOSK. ICE TEA 330ml PUSZK,5900497610339,0.0,18.0,1.75,18.0
Solo Chrupkupki Zielona cebulka,5904918000372,6.0,24.0,2.42,18.0
